In [40]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
import numpy as np

In [134]:
from multiproduct_dashboard import read_dataset, export_dataset

In [131]:
d2 = read_dataset('test.txt')

In [89]:
export_dataset(dataset, 'test.txt')

In [41]:
dataset

Dataset(nodes=[2, 3, 2, 8], supplies=[[230, 170, 100], [210, 190, 100], [150, 250, 100]], demands=[[50, 50, 60, 40, 40, 90, 30, 140, 0], [50, 60, 50, 100, 20, 50, 40, 130, 0], [50, 40, 90, 40, 60, 80, 30, 110, 0]], costs=[array([[963, 810, 919],
       [588, 739, 650]]), array([[437, 971],
       [740, 878],
       [572, 250]]), array([[514, 397, 710, 362, 863, 243, 445, 723],
       [671, 980, 101, 996, 403, 353, 751, 552]])], capacities=[[590, 610], [489, 515, 665], [719, 936], [150, 150, 200, 180, 120, 220, 100, 380]])

In [4]:
from multiproduct import generate_dataset

In [110]:
np.array(dataset.supplies)

array([[230, 170, 100],
       [210, 190, 100],
       [150, 250, 100]])

In [117]:
np.sum(dataset.supplies, 1)

array([500, 500, 500])

In [116]:
supplies

[[230, 170, 100], [210, 190, 100], [150, 250, 100]]

In [115]:
n_nodes = 15
n_levels = 4
n_items = 3

nodes, supplies, demands, costs, capacities = generate_dataset(
            n_nodes, n_levels, total_supplies=[400]*n_items, 
            total_demands=[500]*n_items, transp_costs=(100,1000), 
            random_state=42)        

In [90]:
# %config InlineBackend.close_figures=True
from multiproduct_dashboard import run_multiobjective_GA
# plt.ion()

In [23]:
from multiproduct_dashboard import run_multiobjective_GA
import ipywidgets as widgets

In [8]:
vbox_layout = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    justify_content='space-around',
                    width='100%',
                    max_width='1000px',
                    margin='0 auto',
                    border='solid')

In [9]:
import os
import time
from functools import partial
from collections import namedtuple

In [32]:
Dataset = namedtuple('Dataset', ['nodes', 'supplies', 'demands', 'costs', 'capacities'])
dataset = None
population = None

In [25]:
def gen_dataset(e, n_nodes, n_levels, n_items):
    global dataset
    nodes, supplies, demands, costs, capacities = generate_dataset(
            n_nodes, n_levels, total_supplies=[400]*n_items, 
            total_demands=[500]*n_items, transp_costs=(100,1000), 
            random_state=42)   
    dataset = Dataset(nodes, supplies, demands, costs, capacities)
    generate_dataset_btn.description = 'Generated dataset!'
    time.sleep(0.8)
    generate_dataset_btn.description = 'Generate dataset'

In [26]:
def load_dataset(e):
    pass

In [27]:
def run_genetic_algorithm(e, pop_size, n_gen, early_stopping_rounds):
    global dataset
    global population
    pop, hof, log, toolbox = run_multiobjective_GA(dataset.nodes, 
                            dataset.supplies, dataset.demands,
                            dataset.costs, dataset.capacities, 
                            pop_size=pop_size, n_generations=n_gen,
                            print_log=False, plot_log=True, 
                            early_stopping_rounds=early_stopping_rounds,
                            plot_pop=True, plot_fairness=True,
                            log_output=log_output, pop_output=pop_output,
                            fairness_output=fairness_output)
    

In [28]:

### MENU

space = widgets.Box(layout=widgets.Layout(width='80px'))

#### GENERATE DATASET

n_nodes_input = widgets.IntSlider(15, 5, 30, 1, description='# Nodes')
n_levels_input = widgets.IntSlider(4, 3, 8, 1, description='# Levels')
n_items_input = widgets.IntSlider(3, 1, 7, 1, description='# Items')
generate_dataset_btn = widgets.Button(description='Generate dataset')

aux_box1 = widgets.HBox([n_nodes_input, space, n_levels_input])
aux_box2 = widgets.HBox([n_items_input, space, generate_dataset_btn])

generate_dataset_box = widgets.VBox([aux_box1, aux_box2])

generate_dataset_btn.on_click(partial(gen_dataset, 
                                      n_nodes=n_nodes_input.value,
                                      n_levels=n_levels_input.value,
                                      n_items=n_items_input.value))

#### LOAD DATASET

datasets = {file[:-4]: file \
                   for file in sorted(os.listdir('datasets/evaluation')) \
                   if file.endswith('.txt')}
dataset_dropdown = widgets.Dropdown(
    options = datasets,
    description='Dataset',
    value=datasets[sorted(list(datasets))[0]]
)

load_dataset_btn = widgets.Button(description='Load Dataset')
load_dataset_box = widgets.HBox([dataset_dropdown, space, load_dataset_btn])

load_dataset_btn.on_click(load_dataset)

### GENETIC ALGORITHM

population_size_input = widgets.IntSlider(100, 50, 300, 10, description='Population size',
                          style={'description_width': 'initial'})
n_generations_input = widgets.IntSlider(50, 1, 300, 1, description='# Generations',
                          style={'description_width': 'initial'})
early_stopping_rounds_input = widgets.IntSlider(10, 1, 50, 1, description='Early stopping rounds',
                          style={'description_width': 'initial'})
run_genetic_algorithm_btn = widgets.Button(description='Run Genetic Algorithm',
                                      style={'description_width': 'initial'},
                                      layout=widgets.Layout(width='20%'))
export_ga_solution_btn = widgets.Button(description='Export solutions')

aux_box3 = widgets.HBox([population_size_input, n_generations_input, early_stopping_rounds_input])
aux_box4 = widgets.HBox([run_genetic_algorithm_btn, space, export_ga_solution_btn])

run_genetic_algorithm_btn.on_click(partial(run_genetic_algorithm,
                                           pop_size=population_size_input.value,
                                           n_gen=n_generations_input.value,
                                           early_stopping_rounds=early_stopping_rounds_input.value))

#### PLOTS

log_output = widgets.Output() 
pop_output = widgets.Output()
fairness_output = widgets.Output()

plots_hbox_layout = widgets.Layout(display='flex', flex_flow='row',
                    align_items='center', justify_content='space-around',
                    width='100%', max_width='800px', margin='0 auto')

plots_vbox_layout = widgets.Layout(display='flex', flex_flow='column',
                    align_items='center', justify_content='space-around',
                    width='100%', max_width='800px', margin='0 auto')

pop_fairness_box = widgets.HBox([pop_output, fairness_output], layout=plots_hbox_layout)
plots_box = widgets.VBox([log_output, pop_fairness_box], layout=plots_vbox_layout)

ga_tab = widgets.VBox([aux_box3, aux_box4, plots_box], layout=widgets.Layout(width='900px'))

### SOLVER

solver_desc = widgets.Text(description='Solver')
solver_tab = widgets.VBox([solver_desc], layout=widgets.Layout(width='900px'))

## DASHBOARD

menu = widgets.VBox([generate_dataset_box, load_dataset_box])
tabs = widgets.Tab([ga_tab, solver_tab])
tabs.set_title(0, 'Genetic Algorithm')
tabs.set_title(1, 'Solver')
dashboard = widgets.VBox([menu,
                          tabs], layout=vbox_layout)

In [29]:
dashboard

VBox(children=(VBox(children=(VBox(children=(HBox(children=(IntSlider(value=15, description='# Nodes', max=30, min=5), Box(layout=Layout(width='80px')), IntSlider(value=4, description='# Levels', max=8, min=3))), HBox(children=(IntSlider(value=3, description='# Items', max=7, min=1), Box(layout=Layout(width='80px')), Button(description='Generate dataset', style=ButtonStyle()))))), HBox(children=(Dropdown(description='Dataset', options={'test': 'test.txt'}, value='test.txt'), Box(layout=Layout(width='80px')), Button(description='Load Dataset', style=ButtonStyle()))))), Tab(children=(VBox(children=(HBox(children=(IntSlider(value=100, description='Population size', max=300, min=50, step=10, style=SliderStyle(description_width='initial')), IntSlider(value=50, description='# Generations', max=300, min=1, style=SliderStyle(description_width='initial')), IntSlider(value=10, description='Early stopping rounds', max=50, min=1, style=SliderStyle(description_width='initial')))), HBox(children=(Button(description='Run Genetic Algorithm', layout=Layout(width='20%'), style=ButtonStyle()), Box(layout=Layout(width='80px')), Button(description='Export solution', style=ButtonStyle()))), VBox(children=(Output(), HBox(children=(Output(), Output()), layout=Layout(align_items='center', display='flex', flex_flow='row', justify_content='space-around', margin='0 auto', max_width='800px', width='100%'))), layout=Layout(align_items='center', display='flex', flex_flow='column', justify_content='space-around', margin='0 auto', max_width='800px', width='100%'))), layout=Layout(width='900px')), VBox(children=(Text(value='', description='Solver'),), layout=Layout(width='900px'))), _titles={'0': 'Genetic Algorithm', '1': 'Solver'})), layout=Layout(align_items='center', border='solid', display='flex', flex_flow='column', justify_content='space-around', margin='0 auto', max_width='1000px', width='100%'))

In [36]:
dataset

Dataset(nodes=[2, 3, 2, 8], supplies=[[230, 170, 100], [210, 190, 100], [150, 250, 100]], demands=[[50, 50, 60, 40, 40, 90, 30, 140, 0], [50, 60, 50, 100, 20, 50, 40, 130, 0], [50, 40, 90, 40, 60, 80, 30, 110, 0]], costs=[array([[963, 810, 919],
       [588, 739, 650]]), array([[437, 971],
       [740, 878],
       [572, 250]]), array([[514, 397, 710, 362, 863, 243, 445, 723],
       [671, 980, 101, 996, 403, 353, 751, 552]])], capacities=[[590, 610], [489, 515, 665], [719, 936], [150, 150, 200, 180, 120, 220, 100, 380]])

In [37]:
dataset.nodes

[2, 3, 2, 8]

In [39]:
dataset.supplies

[[230, 170, 100], [210, 190, 100], [150, 250, 100]]